In [1]:
import pymysql
import pandas as pd
import numpy as np
from functools import reduce
import statsmodels.api as sm
import itertools 

In [8]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [9]:
dict = pd.read_excel(r"C:\Users\13670\Desktop\1\泌尿系结石dict.xlsx")
id_list = dict.loc[:,'id'].values.tolist()

tag_df = pd.DataFrame(columns=['ORDER_CODE', 'TAG_ID'])
qn_q_df = pd.DataFrame(columns=['ORDER_CODE', 'SD_CODE', 'abnormal'])

for items in id_list:
    id_str = "'"+ items + "'"
    if ('OBDIM')in id_str:
        tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM T66_APOLLO_ANALYSIS_TAG \
    WHERE SOURCE_TYPE = 2 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
        tag = pd.read_sql(tagsql,con=conn)
        tag_df = pd.concat([tag_df,tag],axis = 0)        
    elif ('N')in id_str:
        qnsql = 'SELECT ORDER_CODE,SD_CODE,\
    IF (IS_ABNORMAL = TRUE, 1,0) AS abnormal \
    FROM T66_ATHENA_BASIC_QUANTITY_RECORD \
    WHERE SZ_STATUS = 1 AND SD_CODE IN ('+id_str+')'
        qn = pd.read_sql(qnsql,con=conn)
        qn_q_df = pd.concat([qn_q_df,qn],axis = 0)
    else :
        qsql = 'SELECT ORDER_CODE,SD_CODE,\
    IF (IS_ABNORMAL = TRUE, 1,0) AS abnormal \
    FROM T66_ATHENA_BASIC_QUALITY_RECORD \
    WHERE SZ_STATUS = 1 AND SD_CODE IN ('+id_str+')'
        q = pd.read_sql(qsql,con=conn)
        qn_q_df = pd.concat([qn_q_df,q],axis = 0)
        
qn_q_duplicates = qn_q_df.drop_duplicates(subset=['ORDER_CODE', 'SD_CODE'])
final_qn_q_pivot = qn_q_duplicates.pivot(index='ORDER_CODE', columns='SD_CODE', values='abnormal')


final_tag_df_pivot = pd.DataFrame(columns=[])
tag_df["score"] = 1
tag_df_pivot = tag_df.pivot(index='ORDER_CODE',columns='TAG_ID',values='score')
for i in tag_df_pivot.loc[:,tag_df_pivot.columns.str.contains('EX')].columns:
    df = tag_df_pivot.loc[tag_df_pivot[i].notnull(),tag_df_pivot.columns.str.contains(i[:9])].fillna(0)
    final_tag_df_pivot = pd.merge(final_tag_df_pivot,df,left_index=True,right_index=True,how='outer')
final_tag_df_pivot    

,OBDIM0006EX,OBDIM0006Tag0001,OBDIM0006Tag0002,OBDIM0006Tag0003,OBDIM0006Tag0004,OBDIM0006Tag0005,OBDIM0006Tag0006,OBDIM0006Tag0007,OBDIM0006Tag0008,OBDIM0006Tag0009,...,OBDIM0086Tag0007,OBDIM0086Tag0008,OBDIM0086Tag0010,OBDIM0086Tag0011,OBDIM0086Tag0013,OBDIM0086Tag0015,OBDIM0086Tag0016,OBDIM0086Tag0017,OBDIM0086Tag0018,OBDIM0086Tag0019
ORDER_CODE,,,,,,,,,,,,,,,,,,,,,
000003745904,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004527245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004598254,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006786795,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000007081060,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999986953057,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999988810200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999990770784,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
person = pd.read_sql('SELECT ORDER_CODE,EXAMINEE_GENDER,EXAMINEE_AGE FROM T66_ATHENA_BASIC_RAW_PERSON_INFO WHERE SZ_STATUS =1',con=conn)
person.replace({'MW':0,'UW':0},inplace=True)
person.replace('M',1,inplace=True)
personp = person.set_index('ORDER_CODE')

In [11]:
dfs = [final_tag_df_pivot,final_qn_q_pivot,personp]
df_total_p1 = reduce(lambda left, right: pd.merge(left, right, on = "ORDER_CODE",how="outer"), dfs)

In [12]:
cols = [c for c in df_total_p1.columns if ("EX") not in c]
df_total_p2 = df_total_p1[cols]

In [13]:
name = dict.set_index('id')['name'].to_dict()
df_total_p2.rename(columns=name,inplace=True)

C:\Users\13670\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
df_total_p3 = df_total_p2.sum(axis = 1,level = 0,skipna = False)

In [15]:
df_total_p4 = df_total_p3.dropna(subset=['诊断'])

In [16]:
df_total_p4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13264 entries, 000014877605 to 999990770784
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   诊断        13264 non-null  float64
 1   肾等回声      13264 non-null  float64
 2   肾低回声      13264 non-null  float64
 3   肾无回声      13264 non-null  float64
 4   肾混合回声     13264 non-null  float64
 5   肾积水       13264 non-null  float64
 6   肾萎缩       13264 non-null  float64
 7   肾缩小       13264 non-null  float64
 8   肾增大       13264 non-null  float64
 9   肾回声不匀     13264 non-null  float64
 10  肾缺失       13264 non-null  float64
 11  尿结晶       0 non-null      float64
 12  肾囊肿       13264 non-null  float64
 13  肾错构瘤      13264 non-null  float64
 14  肾钙乳症      13264 non-null  float64
 15  肾异常回声     13264 non-null  float64
 16  肾占位       13264 non-null  float64
 17  肾发育异常     13264 non-null  float64
 18  肾钙化灶      13264 non-null  float64
 19  肾回声增粗     13264 non-null  float64
 20  甲状腺功能亢进   0 non

In [32]:
df_total_p5 = df_total_p4.drop(columns = ["尿结晶","氨基酸代谢异常","甲状腺功能亢进","尿红细胞","尿路感染","钙代谢异常","镁代谢异常","尿维C"])

In [36]:
df_total_p5.dropna(axis = 0,how = "any")
snz = df_total_p5.applymap(lambda x:1 if x >=1 else 0)

In [34]:
df_final.to_excel(r"C:\Users\13670\Desktop\1\泌尿系结石分析.xlsx")

In [63]:
list = ["诊断","尿pH","尿酸","肾钙乳症","肾发育异常","膀胱息肉"]
snz = snz[list]

In [80]:
resultsnz=pd.DataFrame(index=snz.loc[:,snz.columns!='诊断'].columns)
pred=pd.DataFrame(index=['precision','recall'])
resultsnz=pd.concat([resultsnz,pred])
snz["intercept"]=1
data = snz

In [81]:
for j in range(3,4):
    for i in itertools.combinations(data.loc[:,(data.columns!='诊断')&(data.columns!='intercept')].columns, j):
        try :         
            predictcols=list(i)           
            predictcols.append('intercept')           
            model = sm.Logit(data['诊断'],data[predictcols]).fit()
            data['predict'] = model.predict(data[predictcols]).apply(lambda x: 1 if x>0.5 else 0)           
            if model.pvalues[model.pvalues> 0.05].count()==0 :
                n=np.exp(model.params)         
                p=pd.DataFrame(n,columns=[i])  
                precision=metrics.precision_score(data['诊断'], data['predict'],average='binary')
                recall=metrics.recall_score(data['诊断'], data['predict'],average='binary')               
                f1=metrics.f1_score(data['诊断'], data['predict'], average='binary')
                p.loc['precision',i]=precision               
                p.loc['recall',i]=recall
                p.loc['f1',i]=f1
                resultdata=resultdata.join(p) 
        except Exception:
            pass

In [79]:
resultdata

NameError: name 'resultdata' is not defined

In [2]:
df_total_p7 = df_total_p4.drop(columns = ["尿红细胞","血糖"]).dropna(axis = 0,how = "any")

NameError: name 'df_total_p4' is not defined

In [59]:
df_total_p6 = df_total_p5.dropna(axis = 0,how = "any")

In [62]:
df_total_p7.info()

<class 'pandas.core.frame.DataFrame'>
Index: 346110 entries, 000003745904 to 999994398702
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   肝无回声    346110 non-null  float64
 1   诊断      346110 non-null  float64
 2   肝结节     346110 non-null  float64
 3   肾无回声    346110 non-null  float64
 4   肾功能     346110 non-null  float64
dtypes: float64(5)
memory usage: 15.8+ MB


In [1]:
test = df_total_p7.loc[10000,]

NameError: name 'df_total_p7' is not defined

In [ ]:
import pymysql
import pandas as pd
import numpy as np
from functools import reduce

dict = pd.read_excel(r"C:\Users\13670\Desktop\1\dict_test.xlsx")
id_list = dict.loc[:,'id'].values.tolist()

tag_df = pd.DataFrame(columns=['ORDER_CODE', 'TAG_ID'])
qn_q_df = pd.DataFrame(columns=['ORDER_CODE', 'SD_CODE', 'abnormal'])

for items in id_list:
    id_str = "'"+ items + "'"
    if ('OBDIM')in id_str:
        tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM T66_APOLLO_ANALYSIS_TAG \
    WHERE SOURCE_TYPE = 2 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
        tag = pd.read_sql(tagsql,con=conn)
        tag_df = pd.concat([tag_df,tag],axis = 0)        
    elif ('N')in id_str:
        qnsql = 'SELECT ORDER_CODE,SD_CODE,\
    IF (IS_ABNORMAL = TRUE, 1,0) AS abnormal \
    FROM T66_ATHENA_BASIC_QUANTITY_RECORD \
    WHERE SZ_STATUS = 1 AND SD_CODE IN ('+id_str+')'
        qn = pd.read_sql(qnsql,con=conn)
        qn_q_df = pd.concat([qn_q_df,qn],axis = 0)
    else :
        qsql = 'SELECT ORDER_CODE,SD_CODE,\
    IF (IS_ABNORMAL = TRUE, 1,0) AS abnormal \
    FROM T66_ATHENA_BASIC_QUALITY_RECORD \
    WHERE SZ_STATUS = 1 AND SD_CODE IN ('+id_str+')'
        q = pd.read_sql(qsql,con=conn)
        qn_q_df = pd.concat([qn_q_df,q],axis = 0)
        
person = pd.read_sql('SELECT ORDER_CODE,EXAMINEE_GENDER,EXAMINEE_AGE FROM T66_ATHENA_BASIC_RAW_PERSON_INFO WHERE SZ_STATUS =1',con=conn)
person.replace({'MW':0,'UW':0},inplace=True)
person.replace('M',1,inplace=True)
personp = person.set_index('ORDER_CODE')
        
qn_q_duplicates = qn_q_df.drop_duplicates(subset=['ORDER_CODE', 'SD_CODE'])
final_qn_q_pivot = qn_q_duplicates.pivot(index='ORDER_CODE', columns='SD_CODE', values='abnormal')

final_tag_df_pivot = pd.DataFrame(columns=[])
tag_df["score"] = 1
tag_df_pivot = tag_df.pivot(index='ORDER_CODE',columns='TAG_ID',values='score')
for i in tag_df_pivot.loc[:,tag_df_pivot.columns.str.contains('EX')].columns:
    df = tag_df_pivot.loc[tag_df_pivot[i].notnull(),tag_df_pivot.columns.str.contains(i[:9])].fillna(0)
    final_tag_df_pivot = pd.merge(final_tag_df_pivot,df,left_index=True,right_index=True,how='outer')
final_tag_df_pivot    

dfs = [final_tag_df_pivot,final_qn_q_pivot,personp]
df_total_p1 = reduce(lambda left, right: pd.merge(left, right, on = "ORDER_CODE",how="inner"), dfs)
cols = [c for c in df_total_p1.columns if ("EX") not in c]
df_total_p2 = df_total_p1[cols]
name = dict.set_index('id')['name'].to_dict()
df_total_p2.rename(columns=name,inplace=True)
df_total_p3 = df_total_p2.sum(axis = 1,level = 0,skipna = False)
df_total_p3.applymap(lambda x:1 if x >=1 else 0)
df_total_p4 = df_total_p3.dropna(subset=['诊断'])

# 根据p4的结果决定去掉的列
df_total_final = df_total_p4.drop(columns = ["尿红细胞","血糖"]).dropna(axis = 0,how = "any")